## Create the specification for the Frank Lab NWB extension

Accurately representing the context of an experiment is essential for interpreting behavioral and neural data. The primary purpose of the Frank Lab extension is to represent experimental apparatuses in which our animals perform experiments (i.e. tracks, mazes, sleep boxes, open field arenas). We also provide an extension to represent the behavioral tasks that our animals are performing (i.e. free exploration, W-alternation, sleep).

This notebook creates the blueprint for what a Frank Lab extension must look like.  However, it does not actually implement the extension in Python or Matlab for you to use with your data. Think of building a house; you need the blueprints before you actually build the house.  This notebook is for creating those blueprints, which we call NWB specifications. The NWB specifications will be saved as a pair of YAML (.yaml) files.

For overview documentation on extensions in PyNWB, see the [Extensions Short Tutorial](https://pynwb.readthedocs.io/en/latest/tutorials/general/extensions.html) and [Extensions Long Tutorial](https://pynwb.readthedocs.io/en/latest/extensions.html#extending-nwb). Those tutorials give a basic overview, but the nitty-gritty of working with NWB Extensions is best understood through examples. We therefore recommend reading the documentation and comments in this notebook and in fl_extension.py.

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import networkx as nx
from datetime import datetime
import matplotlib.pyplot as plt

import pynwb
from pynwb import register_class, load_namespaces
from pynwb.spec import NWBNamespaceBuilder, NWBGroupSpec, NWBDatasetSpec, NWBAttributeSpec, NWBLinkSpec
from pynwb.file import MultiContainerInterface, NWBContainer, NWBDataInterface
from hdmf.utils import docval

### Set the name of our extension
This notebook will create two YAML (.yaml) files at the locations defined in the 'ns_path' and 'ext_path' variables below.

In [ ]:
name = 'franklab'
ns_path = name + ".namespace.yaml"
ext_path = name + ".extensions.yaml"

### Create the specification (i.e. draw up the "blueprints") using PyNWB helpers
Now we will draw the blueprints for the Frank Lab extension. We primarily use the [NWBGroupSpec](https://pynwb.readthedocs.io/en/stable/pynwb.spec.html#pynwb.spec.NWBGroupSpec) class to help us create valid NWB groups. An NWB group is basically just a container that can have things like a name, attributes, and data. Like the walls in a house, groups often form the outline of an extension. In fl_extension.py (where we implement this extension in Python), each of these groups will get its own Python class.

We also use the [NWBAttributeSpec]() and [NWBDatasetSpec]() classes to consruct our blueprint. An NWB attribute is just what it sounds like: a short piece of metadata defining some attribute, such as a "help" text. An NWB dataset is also pretty self-explanatory: it's just some data (numbers, text, etc.).  In structuring our blueprint, we often put attributes and datasets inside a group, like putting lights and appliances in a room of a house.

In [1]:
# We first create an NWB attribute for a help string, because it is a required attributed of NWB groups. 
help_attr = NWBAttributeSpec(name='help', doc='help doc', dtype='text', value='help value')



# ---------------------------
# Task (i.e. free exploration, W-alternation, sleep)
# ------
# A Task consists simply of two attributes:
# - a name (i.e. W-alternation, sleep, free exploration)
# - a description of the task
#
# Note that Task inherits from something called NWBDataInterface. NWBDataInterface is a group in PyNWB
# representing basically any kind of data, and which we can store in the NWB file in a ProcessingModule.
# See create_franklab_nwbfile.ipynb for a discussion of Processing Modules.
# ---------------------------
task = NWBGroupSpec(neurodata_type_def='Task',              # define the new Task "neurodata_type"
                    neurodata_type_inc='NWBDataInterface',  # inherit from NWBDataInterface
                    doc='a behavioral task',
                    attributes=[NWBAttributeSpec(name='name', doc='the name of this task', dtype='text'),
                                NWBAttributeSpec(name='description', doc='description of this task', dtype='text'),
                                help_attr])


# ---------------------------
# Apparatus (i.e. tracks, mazes, sleep boxes, arenas)
# --------
# We represent the topology of apparatuses using a graph representation (i.e. nodes and edges) 
# - Nodes represent a component of an apparatus that you'd like the ability to refer
#   (e.g. W-track arms, reward wells, novel object, open field components)
# - Edges represent the topological connectivity of the nodes
#   (i.e. there should be an edge between the left track arm and the left reward well)
# In addiiton, all nodes will contain x/y coordinates that allow us to reconstruct not just
# the topology, but also the spatial geometry, of the apparatuses.
#
# Below, we will first define the nodes and edges. Finally we will define the Apparatus itself
# as a container that holds the nodes and edges as sub-groups.
# ---------------------------

# Node
# -----
# Abstract represention for any kind of node in the topological graph
# We won't actually implement abstract nodes. Rather this is a parent group from which our more 
# specific types of nodes will inherit. Note that NWB specifications have inheritance.
# The quantity '*' means that we can have any number (0 or more) nodes.
node = NWBGroupSpec(neurodata_type_def='Node',
                    neurodata_type_inc='NWBDataInterface',
                    doc='nodes in the graph',
                    quantity='*',
                    attributes=[NWBAttributeSpec('name', 'the name of this node', 'text'),
                                help_attr])

# Edge
# -------
# Edges between any two nodes in the graph.
# An edge's only dataset is the name (string) of the two nodes that the edge connects
# Note that we don't actually include the nodes themselves, just their names, in an edge.
edge = NWBGroupSpec(neurodata_type_def='Edge',
                    neurodata_type_inc='NWBDataInterface',
                    doc='edges in the graph',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='names of the nodes this edge connects',
                                  name='edge_nodes',
                                  dtype='text',
                                  dims=['first_node_name|second_node_name'],
                                  shape=[2])],
                    attributes=[help_attr])

# Point Node
# -----------
# A node that represents a single 2D point in space (e.g. reward well, novel object location)
point_node = NWBGroupSpec(neurodata_type_def='PointNode',
                    neurodata_type_inc='Node',
                    doc='node representing a point in 2D space',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinate of this 2D point',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=[1, 2])],
                    attributes=[help_attr])

# Segment Node
# -------------
# A node that represents a linear segement in 2D space, defined by its start and end points
# (e.g. a single arm of W-track maze)
segment_node = NWBGroupSpec(neurodata_type_def='SegmentNode',
                    neurodata_type_inc='Node',
                    doc='node representing a 2D linear segment defined by its start and end points',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinates of the start and end points of this segment',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=[2, 2])],
                    attributes=[help_attr])

# Polygon Node
# -------------
# A node that represents a polygon area (e.g. open field, sleep box) 
# A polygon is defined by its external vertices and, optionally, by
# any interior points of interest (e.g. interior wells, objects)
polygon_node = NWBGroupSpec(neurodata_type_def='PolygonNode',
                    neurodata_type_inc='Node',
                    doc='node representing a 2D polygon area',
                    quantity='*',
                    datasets=[NWBDatasetSpec(doc='x/y coordinates of the exterior points of this polygon',
                                  name='coords',
                                  dtype='float',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=['null', 2]),
                              NWBDatasetSpec(doc='x/y coordinates of interior points inside this polygon',
                                  name='interior_coords',
                                  dtype='float',
                                  quantity='?',
                                  dims=['num_coords', 'x_vals|y_vals'],
                                  shape=['null', 2])],
                    attributes=[help_attr])

# Apparatus
# -------------
# Finally, we define the apparatus itself.  It is has two sub-groups: nodes and edges.
apparatus = NWBGroupSpec(neurodata_type_def='Apparatus',
                         neurodata_type_inc='NWBDataInterface',
                         doc='a graph of nodes and edges', 
                         quantity='*',
                         groups=[node, edge],
                         attributes=[NWBAttributeSpec(name='name', doc='the name of this apparatus', dtype='text'),
                                     help_attr])


### Save the extension specification
The specification consists of two YAML (.yaml) files: one for the actual blueprint and one for the namespace. In a world with many blueprints, we need namespaces to effectively categorize/store them, like the drawers of an architect's filing cabinet.

In [1]:
ns_builder = NWBNamespaceBuilder(name + ' extensions', name)
ns_builder.add_spec(ext_path, apparatus)
ns_builder.add_spec(ext_path, task)
ns_builder.add_spec(ext_path, point_node)
ns_builder.add_spec(ext_path, segment_node)
ns_builder.add_spec(ext_path, polygon_node)
ns_builder.export(ns_path)

NameError: name 'NWBAttributeSpec' is not defined